<h1>Purpose </h1>
This notebook serves as a submission for the IBM professional data science certificate capstone project "Battle of the neighborhoods"


# Begin with imports and data acquisition

In [ ]:
import pandas as pd
import numpy as np
data_source = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

data= pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

data.head(15)


## Clean data to much criteria:
* Only process cells that have an assigned borough, ignore others.
* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma.
* If the neighborhood is unassigned, assign it to be the same as the Borough


#### Begin by removing missing borough values

In [ ]:
#Remove values where borough is unassigned
#Begin by replacing not assigned with NaT
data['Borough'].replace("Not assigned",pd.NaT,regex=True,inplace=True)
#Now drop NaT values in Borough
data.dropna(subset = ['Borough'],inplace=True)
data.reset_index(inplace=True)
data

#### Handle missing neighborhoods

In [ ]:
data['Neighborhood'].replace(np.NaN,data.Neighborhood,regex=True,inplace=True)

data = data[['Postal code' , 'Borough' ,'Neighborhood']]
data

#### Group by postal code and seperate by coloumns

In [ ]:
#Remove space from Postal code
data.columns
column_names=['Postal_code', 'Borough', 'Neighborhood']
data.columns=column_names
data.columns


In [ ]:
counts = data.Postal_code.value_counts()
counts[counts>1]
data = data.groupby('Borough').apply(lambda x: x.apply(lambda y:  ', '.join(y) \
                                                   if y.name =='Neighbourhood' \
                                                   else y.tolist()[0]))
data.reset_index(inplace=True, drop=True)
data

In [ ]:
#Replace / with ,
data['Neighborhood'] = data['Neighborhood'].str.replace('/',',')
data

## Next step is to add a lattitude and longitude value for each Postal code

### Data has been downloaded to csv file for convenience and reliability

In [ ]:
#First load the data into a dictionary of the following format:
# {postal_code:(lat,long)}
import csv
GeoDict={}
reader = csv.DictReader(open('Geospatial_Coordinates.csv'))
for row in reader:

    code=row["Postal Code"]
    latlong=(row["Latitude"],row["Longitude"])
    GeoDict.update({code:latlong})
    


### Then the coordinates are prepared to be added to the DF

#### This is done by iterating over the postal codes in the DF and looking up their values in a dictionary

In [ ]:
#First expand the data frame
data["Latitude"] = ""
data["Longitude"]= ""


In [ ]:
for index,row in data.iterrows():
    row["Latitude"],row["Longitude"]=GeoDict[row["Postal_code"]]

In [ ]:
data.head()

## Draw the map based on the data frame

In [ ]:
#Begin with imports
from geopy.geocoders import Nominatim
import folium
#Then begin setup
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Torronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(data['Latitude'], fata['Longitude'], data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_toronto